In [4]:
!$JAVA_HOME

!pip list | grep spark
!echo "from pyspark.sql import SparkSession; print('lol')" | python 2>&1

pyspark           3.2.1
spark-nlp         4.2.0
lol


In [8]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

# Start SparkSession with Spark NLP
# start() functions has 3 parameters: gpu, m1, and memory
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(m1=True) will start the session with macOS M1 support
# sparknlp.start(memory="16G") to change the default driver memory in SparkSession
spark = sparknlp.start(memory="2G")

# Download a pre-trained pipeline
pipeline = PretrainedPipeline('explain_document_dl', lang='en')

# Your testing dataset
text = """
The Mona Lisa is a 16th century oil painting created by Leonardo.
It's held at the Louvre in Paris.
"""

# Annotate your testing dataset
result = pipeline.annotate(text)

# What's in the pipeline
print(list(result.keys()))
# Output: ['entities', 'stem', 'checked', 'lemma', 'document','pos', 'token', 'ner', 'embeddings', 'sentence']

# Check the results
print(list(result['entities']))
print(result)
# Output: ['Mona Lisa', 'Leonardo', 'Louvre', 'Paris']

explain_document_dl download started this may take some time.


TypeError: 'JavaPackage' object is not callable

In [7]:
from pyspark.sql import SparkSession
from datetime import datetime, date
import sparknlp

from pyspark.sql import Row
import pyspark.sql.functions as F

spark = sparknlp.start()

history = spark.read.csv('sample_history.csv', sep='|', header=True)

print(history)
print(history.schema)

history.show(3)

DataFrame[Kind : string, Query : string,  url: string]
StructType(List(StructField(Kind ,StringType,true),StructField(Query ,StringType,true),StructField( url,StringType,true)))
+---------+--------------------+--------------------+
|    Kind |              Query |                 url|
+---------+--------------------+--------------------+
| Visited |Your data in Sear...| https://myaccoun...|
|Searched |timeline of your ...|                    |
|Searched |       cloud of word|                    |
+---------+--------------------+--------------------+
only showing top 3 rows



In [6]:
# https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/2.Text_Preprocessing_with_SparkNLP_Annotators_Transformers.ipynb

from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import *
from sparknlp.base import *

document_assembler = DocumentAssembler()\
 .setInputCol("text")\
 .setOutputCol("document")
sentenceDetector = SentenceDetector()\
 .setInputCols(["document"])\
 .setOutputCol("sentences")
tokenizer = Tokenizer() \
 .setInputCols(["sentences"]) \
 .setOutputCol("token")
normalizer = Normalizer()\
 .setInputCols(["token"])\
 .setOutputCol("normal")
word_embeddings=WordEmbeddingsModel.pretrained()\
 .setInputCols(["document","normal"])\
 .setOutputCol("embeddings")
nlpPipeline = Pipeline(stages=[
 document_assembler, 
 sentenceDetector,
 tokenizer,
 normalizer,
 word_embeddings,
 ])
pipelineModel = nlpPipeline.fit(history)

TypeError: 'JavaPackage' object is not callable

In [ ]:
pipelineModel.transform(history)